# Week 3 — Part 03: Structured Outputs and Validation Lab

**Estimated time:** 90–120 minutes

---

## Pre-study (Self-learn)

Foundations Course assumes Self-learn is complete. If you need a refresher on structured outputs and validation:

- [Foundations Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Structured outputs and schemas](../self_learn/Chapters/3/01_function_calling_structured_outputs.md)

---

## What success looks like (end of Part 03)

- You can parse LLM outputs into structured objects.
- You can validate outputs against schemas.
- You can implement retry logic for invalid outputs.

### Checkpoint

After running this notebook:
- You can parse JSON outputs into Pydantic models
- You can validate and retry on schema failures

## Learning Objectives

- Parse LLM outputs into structured formats
- Validate outputs against schemas
- Implement retry patterns for validation failures

### What this part covers
This notebook builds a **parse → validate → retry/repair** pipeline for LLM outputs.

Even with a strict contract prompt, models sometimes return:
- prose before/after the JSON
- single quotes instead of double quotes
- missing required keys
- extra keys

This notebook shows how to detect each failure mode and repair it automatically — with a hard retry cap so you don't loop forever.

## Overview

Models can produce valid JSON, or almost-JSON (extra prose, single quotes, trailing commas).

This lab builds a deterministic wrapper:

1. ask for strict JSON
2. parse it
3. validate schema
4. retry/repair on failure (capped)

Key habit: save raw output when parsing/validation fails so debugging is inspection, not guesswork.

If you need more background on schemas/validation, use the Self-learn links at the top of the notebook.

### What this cell does
Defines the two-stage validation pipeline:

- **`parse_json_strict()`** — attempts `json.loads()`. Fails if the text is not valid JSON (e.g., prose before the JSON, single quotes, trailing commas).
- **`validate_shape()`** — checks that the parsed dict has exactly the right keys and types. Fails if keys are missing, extra, or the wrong type.

Then tests both against 3 common bad outputs:
1. Prose before the JSON (`"Here is the JSON: {...}"`) — parse fails
2. Single quotes (`{'person': ...}`) — parse fails (Python dict syntax ≠ JSON)
3. Missing key (`{"person": "Ada"}`) — shape validation fails

**Key insight:** Parse failure and schema failure have different root causes. Parse failure = the model ignored the "return ONLY JSON" instruction. Schema failure = the model returned valid JSON but with the wrong structure. Different fixes for each.

In [ ]:
import json
from pathlib import Path


def parse_json_strict(text: str) -> dict:
    data = json.loads(text)
    if not isinstance(data, dict):
        raise ValueError("expected a JSON object")
    return data


def validate_shape(data: dict) -> None:
    allowed = {"person", "company"}
    extra = set(data.keys()) - allowed
    missing = allowed - set(data.keys())
    if missing:
        raise ValueError(f"missing keys: {sorted(missing)}")
    if extra:
        raise ValueError(f"extra keys: {sorted(extra)}")

    for k in ["person", "company"]:
        v = data[k]
        if v is not None and not isinstance(v, str):
            raise ValueError(f"{k} must be string or null")


print(validate_shape(parse_json_strict('{"person": "Ada", "company": null}')))

OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

### What this cell does
Defines `call_llm_stub()` — a simulated LLM that always returns the "prose + JSON" failure pattern on the first call, then returns clean JSON when the prompt contains "REPAIR".

**Why simulate failures?** You need to test your retry logic without burning real API credits. This stub lets you exercise the full repair loop deterministically.

**What to notice:** The stub returns `"Here is the JSON: {...}"` — a very common failure where the model adds explanatory text before the JSON. Your parser must handle this gracefully.

In [ ]:
def parse_and_validate(text: str) -> dict:
    data = parse_json_strict(text)
    validate_shape(data)
    return data


bad_outputs = [
    "Here is the JSON: {\"person\": \"Ada\", \"company\": null}",
    "{'person': 'Ada', 'company': null}",
    '{"person": "Ada"}',
]

for raw in bad_outputs:
    try:
        parse_and_validate(raw)
        print("OK", raw)
    except Exception as e:
        print("FAIL", type(e).__name__, "->", str(e))

### What this cell does
Implements `extract_with_repair()` — the full retry/repair loop:

1. Build the base prompt
2. Call the LLM
3. Try to parse and validate the output
4. If it fails: build a **repair prompt** that tells the model exactly what went wrong, then retry
5. Cap retries at `max_retries` to avoid infinite loops

**Key design:** The repair prompt includes the invalid output AND the error message. This gives the model concrete feedback: "your output was `{...}` and the error was `missing keys: ['company']`." Vague repair prompts ("try again") are much less effective.

**Your task:** Implement `extract_with_repair_todo()` with two improvements: (1) save the raw output to disk on failure so you can inspect it, and (2) distinguish parse errors from schema errors in the repair message.

In [ ]:
def call_llm_stub(prompt: str) -> str:
    # Simulate a model that sometimes returns almost-JSON.
    if "REPAIR" in prompt:
        return '{"person": "Ada Lovelace", "company": null}'
    return "Here is the JSON: {\"person\": \"Ada Lovelace\", \"company\": null}"

In [ ]:
def extract_with_repair(text: str, call_llm, *, max_retries: int = 2) -> dict:
    base_prompt = (
        "Return ONLY JSON with keys person, company (null when unknown).\n"
        f"INPUT:\n{text}\n"
    )

    prompt = base_prompt
    last_err = None
    for attempt in range(max_retries + 1):
        raw = call_llm(prompt)
        try:
            return parse_and_validate(raw)
        except Exception as e:
            last_err = str(e)
            prompt = (
                "REPAIR: Your previous output was invalid.\n"
                "Return ONLY JSON with keys person, company.\n"
                f"Invalid output:\n{raw}\n\n"
                f"Error:\n{last_err}\n"
            )

    raise ValueError(f"Failed after retries. Last error: {last_err}")


print(extract_with_repair("Ada Lovelace", call_llm_stub, max_retries=2))

In [ ]:
def extract_with_repair_todo(text: str, call_llm, *, max_retries: int = 2) -> dict:
    # TODO:
    # - Add raw-output persistence to output/llm_raw.txt on failure.
    # - Separate parse failures from schema failures in the error message.
    # - Keep retries capped (max_retries).
    return extract_with_repair(text, call_llm, max_retries=max_retries)


print("Implement extract_with_repair_todo().")
print(extract_with_repair_todo("Ada Lovelace", call_llm_stub, max_retries=2))

## Common pitfalls

- Asking for JSON but not banning extra text
- Not separating parse failure vs schema failure
- No retry cap
- Mixing business logic with parsing/validation

## References

- Python `json`: https://docs.python.org/3/library/json.html
- Pydantic (optional): https://docs.pydantic.dev/latest/
- JSON Schema: https://json-schema.org/
- Tenacity: https://tenacity.readthedocs.io/

## Exercise: Persist raw failures

Implement the TODO function below.

Goal:

- When parsing/validation fails, persist the raw output under `output/`.
- Return the output path so you can reference it in a report/debugging.

Checkpoint:

- Running the cell creates a file like `output/raw_failure.txt`.

In [ ]:
from pathlib import Path


OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)


def persist_raw_failure_todo(raw_text: str, *, filename: str = "raw_failure.txt") -> Path:
    # TODO: implement
    out_path = OUTPUT_DIR / filename
    out_path.write_text("TODO\n", encoding="utf-8")
    return out_path


for raw in bad_outputs:
    try:
        parse_and_validate(raw)
    except Exception:
        p = persist_raw_failure_todo(raw)
        print("saved raw failure to", p)

### Solution: Persist raw failures

Reference implementation for `persist_raw_failure_todo`.

In [ ]:
def persist_raw_failure_todo(raw_text: str, *, filename: str = "raw_failure.txt") -> Path:
    out_path = OUTPUT_DIR / filename
    out_path.write_text(raw_text, encoding="utf-8")
    return out_path


for raw in bad_outputs:
    try:
        parse_and_validate(raw)
    except Exception:
        p = persist_raw_failure_todo(raw)
        print("saved raw failure to", p)

In [ ]:
def parse_and_validate(text: str) -> dict:
    data = parse_json_strict(text)
    validate_shape(data)
    return data


bad_outputs = [
    "Here is the JSON: {\"person\": \"Ada\", \"company\": null}",
    "{'person': 'Ada', 'company': null}",
    '{"person": "Ada"}',
]

for raw in bad_outputs:
    try:
        parse_and_validate(raw)
        print("OK", raw)
    except Exception as e:
        print("FAIL", type(e).__name__, "->", str(e))

## Appendix: Solutions (peek only after trying)

Reference implementation for `extract_with_repair_todo` that persists raw outputs and clarifies error stages.

In [ ]:
def extract_with_repair_todo(text: str, call_llm, *, max_retries: int = 2) -> dict:
    base_prompt = (
        "Return ONLY JSON with keys person, company (null when unknown).\n"
        f"INPUT:\n{text}\n"
    )

    prompt = base_prompt
    last_err = None
    last_raw = None

    for attempt in range(max_retries + 1):
        raw = call_llm(prompt)
        last_raw = raw
        try:
            data = parse_json_strict(raw)
        except Exception as e:
            last_err = f"PARSE_ERROR: {e}"
            prompt = (
                "REPAIR: Your previous output was invalid JSON.\n"
                "Return ONLY JSON with keys person, company.\n"
                f"Invalid output:\n{raw}\n\n"
                f"Error:\n{last_err}\n"
            )
            continue

        try:
            validate_shape(data)
            return data
        except Exception as e:
            last_err = f"SCHEMA_ERROR: {e}"
            prompt = (
                "REPAIR: Your previous output failed schema validation.\n"
                "Return ONLY JSON with keys person, company.\n"
                f"Invalid output:\n{raw}\n\n"
                f"Error:\n{last_err}\n"
            )

    raw_path = OUTPUT_DIR / "llm_raw.txt"
    raw_path.write_text(last_raw or "", encoding="utf-8")
    raise ValueError(f"Failed after retries. {last_err}. Raw saved to {raw_path}")


try:
    extract_with_repair_todo("Ada Lovelace", call_llm_stub, max_retries=1)
except Exception as e:
    print("expected failure path exercised:", str(e))